# statsmodels

> Functions that extract and standardize results from `statsmodels.api.OLS.fit()` objects and `statsmodels.api.OLS.fit().get_robustcov_results()` objects.

In [ ]:
#|default_exp statsmodels_results

In [ ]:
#|hide 
from nbdev.showdoc import *
from nbdev.test import * 

In [ ]:
#|exports
import numpy as np
import pandas as pd

In [ ]:
import statsmodels.api as sm

Set up an example dataset and run a few regressions to showcase the functions in this module.

In [ ]:
np.random.seed(123)
df = pd.DataFrame(np.random.rand(9,3), 
                  columns=['y','x','z'],
                  index = pd.MultiIndex.from_product([[1,2,3],[1,2,3]], names=['firmid','time'])
                  ).assign(cons = 1)
sm1 = sm.OLS(df['y'], df[['cons','x']]).fit()
sm2 = sm.OLS(df['y'], df[['cons','x','z']]).fit().get_robustcov_results(cov_type='HAC', maxlags=2)

In [ ]:
#|export 
def default_stats():
    """List of statistics calculated in this module"""
    return ['ynames', 'xnames', 'params', 'tstats', 'pvalues', 'covmat', 'se', 'r2', 'nobs']

Each of these statistics is calculated by a separate function in this module (function has the same name as the statistic).

In [ ]:
default_stats()

['ynames',
 'xnames',
 'params',
 'tstats',
 'pvalues',
 'covmat',
 'se',
 'r2',
 'nobs']

The following functions will extract some key information from the `sm1` and `sm2` objects. 

Note that for results produced by `.get_robustcov_results()` (the `sm2` example), many atributes (e.g. `params`, `tstats`, and `pvalues`) are returned as ndarray not pd.Series.\
So we will convert them to pd.Series in the appropriate functions.

These statistics have to have the same name across all '{package_name}_results' modules.

In [ ]:
#|export 
def ynames(res): return list(res.model.endog_names)

In [ ]:
ynames(sm1)

['y']

In [ ]:
#|export 
def xnames(res): return res.model.exog_names

In [ ]:
xnames(sm1)

['cons', 'x']

In [ ]:
#|export 
def params(res): 
    if isinstance(res.params, np.ndarray):
            return pd.Series(res.params, index=xnames(res)) 
    else:
        return res.params

In [ ]:
params(sm1)

cons    0.507852
x       0.345003
dtype: float64

In [ ]:
params(sm2)

cons    0.698578
x       0.571069
z      -0.640497
dtype: float64

In [ ]:
#|export 
def tstats(res):
    if isinstance(res.tvalues, np.ndarray):
            return pd.Series(res.tvalues, index=xnames(res)) 
    else:
        return res.tvalues

In [ ]:
tstats(sm1)

cons    3.905440
x       1.292246
dtype: float64

In [ ]:
tstats(sm2)

cons    21.482641
x        2.848571
z       -3.547016
dtype: float64

In [ ]:
#|export
def pvalues(res):
    if isinstance(res.pvalues, np.ndarray):
            return pd.Series(res.pvalues, index=xnames(res)) 
    else:
        return res.pvalues

In [ ]:
pvalues(sm1)

cons    0.005858
x       0.237293
dtype: float64

In [ ]:
pvalues(sm2)

cons    6.638188e-07
x       2.923636e-02
z       1.211566e-02
dtype: float64

In [ ]:
#|export
def covmat(res):
    if isinstance(res.cov_params(), np.ndarray):
            return pd.DataFrame(res.cov_params(), index=xnames(res), columns=xnames(res)) 
    else:
        return res.cov_params()

In [ ]:
covmat(sm1)

,cons,x
cons,0.016910,-0.030531
x,-0.030531,0.071278


In [ ]:
covmat(sm2)

,cons,x,z
cons,0.001057,0.002529,-0.004205
x,0.002529,0.040190,-0.032507
z,-0.004205,-0.032507,0.032607


In [ ]:
#|export
def se(res): return pd.Series(np.sqrt(np.diag(np.array(covmat(res)))),index=xnames(res))

In [ ]:
se(sm1)

cons    0.130037
x       0.266979
dtype: float64

In [ ]:
se(sm2)

cons    0.032518
x       0.200476
z       0.180574
dtype: float64

In [ ]:
#|export
def nobs(res): return int(res.nobs)

In [ ]:
nobs(sm1)

9

In [ ]:
#|export
def r2(res): return res.rsquared

In [ ]:
r2(sm1)

0.19260886185799486

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()